# Problem Statement

**PROJECTO 1** <br>
**Análisis exploratorio y modelado predictivo de precios de viviendas en Barcelona usando Python y SQL**

## Objective
Desarrollar un análisis completo y un modelo predictivo para los precios de viviendas en Barcelona, utilizando datos extraídos del portal Fotocasa. El objetivo es aplicar técnicas de extracción, manipulación y análisis de datos, así como algoritmos de Machine Learning, para predecir los precios de las viviendas en función de diversas características.

## Data Description
- **price**: The price of the real-state.
- **rooms**: Number of rooms.
- **bathroom**: Number of bathrooms.
- **lift**: whether a building has an elevator (also known as a lift in some regions) or not
- **terrace**: If it has a terrace or not.
- **square_meters**: Number of square meters.
- **real_state**: Kind of real-state.
- **neighborhood**: Neighborhood
- **square_meters_price**: Price of the square meter

## Importing necessary libraries

In [24]:
import pandas as pd
import numpy as np

## Loading the Dataset

In [3]:
df=pd.read_csv('Barcelona_Fotocasa_HousingPrices.csv')

# Data Overview

In [8]:
df.head() # preview a sample first 5 rows

,Unnamed: 0,price,rooms,bathroom,lift,terrace,square_meters,real_state,neighborhood,square_meters_price
0,0,750,3,1,True,False,60,flat,Horta- Guinardo,12.500000
1,1,770,2,1,True,False,59,flat,Sant Andreu,13.050847
2,2,1300,1,1,True,True,30,flat,Gràcia,43.333333
3,3,2800,1,1,True,True,70,flat,Ciutat Vella,40.000000
4,4,720,2,1,True,False,44,flat,Sant Andreu,16.363636


In [6]:
df.tail() # preview a sample last 5 rows

,Unnamed: 0,price,rooms,bathroom,lift,terrace,square_meters,real_state,neighborhood,square_meters_price
8183,8183,1075,2,2,False,False,65,flat,Gràcia,16.538462
8184,8184,1500,3,2,True,False,110,flat,Eixample,13.636364
8185,8185,1500,2,2,True,True,90,flat,Sarria-Sant Gervasi,16.666667
8186,8186,1500,3,2,True,False,110,flat,Eixample,13.636364
8187,8187,1500,3,2,True,False,110,flat,Eixample,13.636364


In [7]:
df.sample(20) # preview a sample random n rows

,Unnamed: 0,price,rooms,bathroom,lift,terrace,square_meters,real_state,neighborhood,square_meters_price
915,915,1100,2,1,False,True,35,flat,Eixample,31.428571
6089,6089,650,1,1,False,False,34,flat,Ciutat Vella,19.117647
3373,3373,750,2,1,True,False,49,flat,Horta- Guinardo,15.306122
3847,3847,900,3,2,True,False,63,flat,Gràcia,14.285714
3075,3075,950,3,2,True,False,100,flat,Sant Martí,9.500000
7820,7820,2200,2,2,True,False,68,flat,Gràcia,32.352941
2269,2269,1350,2,1,True,True,75,attic,Eixample,18.000000
7775,7775,1250,4,2,True,False,95,flat,Sarria-Sant Gervasi,13.157895
1387,1387,1275,3,2,True,True,93,flat,Sarria-Sant Gervasi,13.709677
8054,8054,761,2,1,False,True,59,attic,Sant Andreu,12.898305


In [9]:
print("There are", df.shape[0], 'rows and', df.shape[1], "columns.") # number of observations and features


There are 8188 rows and 10 columns.


In [10]:
df.dtypes # data types

Unnamed: 0               int64
price                    int64
rooms                    int64
bathroom                 int64
lift                      bool
terrace                   bool
square_meters            int64
real_state              object
neighborhood            object
square_meters_price    float64
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           8188 non-null   int64  
 1   price                8188 non-null   int64  
 2   rooms                8188 non-null   int64  
 3   bathroom             8188 non-null   int64  
 4   lift                 8188 non-null   bool   
 5   terrace              8188 non-null   bool   
 6   square_meters        8188 non-null   int64  
 7   real_state           7920 non-null   object 
 8   neighborhood         8188 non-null   object 
 9   square_meters_price  8188 non-null   float64
dtypes: bool(2), float64(1), int64(5), object(2)
memory usage: 527.9+ KB


In [13]:
df.describe(include="all").T # statistical summary of the data.

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,8188.0,NaN,NaN,NaN,4093.5,2363.816335,0.0,2046.75,4093.5,6140.25,8187.0
price,8188.0,NaN,NaN,NaN,1444.092574,1125.886215,320.0,875.0,1100.0,1540.0,15000.0
rooms,8188.0,NaN,NaN,NaN,2.420738,1.138592,0.0,2.0,2.0,3.0,10.0
bathroom,8188.0,NaN,NaN,NaN,1.508793,0.732798,1.0,1.0,1.0,2.0,8.0
lift,8188,2,True,5710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
terrace,8188,2,False,6518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
square_meters,8188.0,NaN,NaN,NaN,84.610161,47.874028,10.0,56.0,73.0,95.0,679.0
real_state,7920,4,flat,6505,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neighborhood,8188,10,Eixample,2401,NaN,NaN,NaN,NaN,NaN,NaN,NaN
square_meters_price,8188.0,NaN,NaN,NaN,17.739121,9.245241,4.910714,12.790698,15.306122,19.444444,186.666667


In [17]:
# Uniques
df.nunique() # Checking for number of variations in the data
for i in df.columns: # Checking uniques
    print (i,": ",df[i].unique())

Unnamed: 0 :  [   0    1    2 ... 8185 8186 8187]
price :  [  750   770  1300  2800   720  1100  1350   900  1165  1050  2500  1335
  2200   980  1500  1080   950   850  1800  2300   842  1180  1200  1400
   852  1250  1000  1600  1450  1246  1580  1950  1060  2700   625  2550
   925  1290  3000   780  1053   650  1190   660  1585  1850   710  3468
  1700   590  3300   700  2900   825  2000  2850  2057  3144  1550  2400
   976  2021  1540  4000   890  1970  1210  1185  2250  1900  3500   795
   895   995  1170   730  1750  1220   800  1195   930   990  1150  1475
  5250   702  1175  1360   690  1690  1130  5720  2290  3600  1480  1835
  2050  3200  3768  2229  2150  1806  2155  2317  2008  2040  2760  1425
  1495  3400  3800  1125   970  8482  3219  1626  1724  5000   935  1590
  5200  2600  1650  6800  6000   975  4900  8000  1975  8500  1949  1385
   550  3833  4600  3850  9075  7000  5300  4700   620   920  1085  1675
  2580   520   840   760 10000  2520  3900  2975  6500  4200  245

In [16]:
# Uniques
cat_cols = df.select_dtypes(include=['category', 'object']).columns.tolist()
for column in cat_cols:
    print(df[column].value_counts())
    print("-" * 50)


real_state
flat         6505
apartment     991
attic         315
study         109
Name: count, dtype: int64
--------------------------------------------------
neighborhood
Eixample               2401
Sarria-Sant Gervasi    1396
Ciutat Vella           1365
Gràcia                  688
Sant Martí              617
Sants-Montjuïc          590
Les Corts               524
Horta- Guinardo         315
Sant Andreu             178
Nou Barris              114
Name: count, dtype: int64
--------------------------------------------------


In [15]:
# Duplicates
df.duplicated().sum() # Checking for duplicate entries in the data

np.int64(0)

# Consolidated notes on Data Overview

- There are 8188 rows and 10 columns.
- The variable 'Unnamed' represent index and should be deleted from data
- Data types are aligned with information
- There is missing data (NaN) on variable 'real_state'. To be replaced by "other"
- There are four types of real states being the most common "flat"
- Most units do not have terrace
- Most units do have lift
- The neighborhood with largest unit count is "Eixample"
- Units size goes from 10m2 to 679m2, with a mean of 84.61m2
- Units prices goes from 320EUR to 15000EUR/month, with mean of 1444EUR/month
- price range is assumed referred to monthly rent, so considered as EUR per month
- Units prices by square meter goes from 4.9EUR/m2/month to 186EUR/m2/month, with mean of 17.7EUR/m2/month
- There are units listed with cero rooms (to be investigated)
- Target variable for modeling is "price"

# Exploratory Data Analysis (EDA)

## EDA Functions

In [18]:
def univariate_numerical(data):
    '''
    Function to generate two plots for each numerical variable
    Histplot for variable distribution
    Boxplot for statistical summary 
    '''
    # Select numerical columns
    numerical_cols = data.select_dtypes(include=[np.number]).columns
    
    # Determine the number of rows and columns
    num_vars = len(numerical_cols)
    num_cols = 4
    num_rows = int(np.ceil(num_vars * 2 / num_cols))
    
    # Create a figure with the specified size
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, num_rows * 5))
    
    # Flatten the axes array for easy iteration
    axes = axes.flatten()
    
    # Plot each variable with a histplot and a boxplot
    for i, col in enumerate(numerical_cols):
        mean_value = data[col].mean()
        
        # Histplot with KDE
        sns.histplot(data[col], kde=True, ax=axes[i*2])
        axes[i*2].axvline(mean_value, color='r', linestyle='--')
        axes[i*2].set_title(f'Distribution of {col}')
        axes[i*2].text(mean_value, axes[i*2].get_ylim()[1]*0.8, f'Mean: {mean_value:.2f}', color='r', va='baseline', ha='left',rotation=90)
        
        # Boxplot
        sns.boxplot(y=data[col], ax=axes[i*2 + 1])
        axes[i*2 + 1].axhline(mean_value, color='r', linestyle='--')
        axes[i*2 + 1].set_title(f'Boxplot of {col}')
        axes[i*2 + 1].text(axes[i*2 + 1].get_xlim()[1]*0.8, mean_value, f'mean: {mean_value:.2f}', color='r', va='baseline', ha='right')
    
    # Hide any remaining empty subplots
    for j in range(num_vars * 2, len(axes)):
        fig.delaxes(axes[j])
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

In [19]:
def univariate_categorical(data):
    '''
    Function to generate countplot for each categorical variable
    Labeled with count and percentage
    '''
    # List of categorical columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Number of columns in the grid
    num_cols = 4
    
    # Calculate the number of rows needed
    num_rows = (len(categorical_columns) + num_cols - 1) // num_cols
    
    # Create the grid
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, num_rows * 5), constrained_layout=True)
    axes = axes.flatten()
    
    # Plot each countplot in the grid
    for i, col in enumerate(categorical_columns):
        ax = axes[i]
        plot = sns.countplot(x=col, data=data, order=data[col].value_counts().index, ax=ax)
        ax.set_title(f'Count of {col}')
           
        # Add total count and percentage annotations
        total = len(data)
        for p in plot.patches:
            height = p.get_height()
            percentage = f'{(height / total * 100):.1f}%'
            plot.text(x=p.get_x() + p.get_width() / 2,
                      y=height + 2,
                      s=f'{height:.0f}\n({percentage})',
                      ha='center')
        
        # Limit x-axis labels to avoid overlap
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Remove any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    # Show the plot
    plt.show()


## Data Preprocesing

- Missing value treatment
- Feature engineering
- Outlier detection and treatment
- Preparing data for modeling
- Any other preprocessing steps

In [48]:
data=df.copy() # Data preprocesing over a copy of original dataset

In [49]:
data.columns #Check column names

Index(['Unnamed: 0', 'price', 'rooms', 'bathroom', 'lift', 'terrace',
       'square_meters', 'real_state', 'neighborhood', 'square_meters_price'],
      dtype='object')

In [50]:
data.isna().sum() # missing values per feature

Unnamed: 0               0
price                    0
rooms                    0
bathroom                 0
lift                     0
terrace                  0
square_meters            0
real_state             268
neighborhood             0
square_meters_price      0
dtype: int64

In [51]:
data['real_state'].value_counts()

real_state
flat         6505
apartment     991
attic         315
study         109
Name: count, dtype: int64

In [52]:
data['real_state']=data['real_state'].replace(np.nan, 'other')

In [53]:
data.isna().sum() # missing values per feature

Unnamed: 0             0
price                  0
rooms                  0
bathroom               0
lift                   0
terrace                0
square_meters          0
real_state             0
neighborhood           0
square_meters_price    0
dtype: int64

In [54]:
data['real_state'].value_counts()

real_state
flat         6505
apartment     991
attic         315
other         268
study         109
Name: count, dtype: int64

In [55]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [56]:
data.head()

,price,rooms,bathroom,lift,terrace,square_meters,real_state,neighborhood,square_meters_price
0,750,3,1,True,False,60,flat,Horta- Guinardo,12.500000
1,770,2,1,True,False,59,flat,Sant Andreu,13.050847
2,1300,1,1,True,True,30,flat,Gràcia,43.333333
3,2800,1,1,True,True,70,flat,Ciutat Vella,40.000000
4,720,2,1,True,False,44,flat,Sant Andreu,16.363636


In [60]:
data.loc[data['rooms']==0].describe().T

,count,mean,std,min,25%,50%,75%,max
price,204.0,956.093137,993.654149,423.000000,600.000000,722.500000,850.0,10750.000000
rooms,204.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
bathroom,204.0,1.122549,0.465226,1.000000,1.000000,1.000000,1.0,4.000000
square_meters,204.0,56.009804,62.074958,15.000000,30.000000,40.000000,58.0,679.000000
square_meters_price,204.0,18.751196,5.291655,8.910256,14.666667,18.169643,22.0,37.333333


In [70]:
data.loc[(data['rooms']==0)&(data['bathroom']==1)].describe().T

,count,mean,std,min,25%,50%,75%,max
price,188.0,890.973404,972.233948,423.000000,600.0,700.000000,825.00000,10750.000000
rooms,188.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000
bathroom,188.0,1.000000,0.000000,1.000000,1.0,1.000000,1.00000,1.000000
square_meters,188.0,50.101064,57.048768,15.000000,30.0,39.000000,53.00000,679.000000
square_meters_price,188.0,19.060333,5.153105,9.166667,15.0,18.421053,22.08057,37.333333


## Univariate Analysis

## Bivariate Analysis

# Consolidated notes on Exploratory Data Analysis

**Functions:**
- univariate_numerical(data): Function to generate two plots for each numerical variable. Histplot for variable distribution. Boxplot for statistical summary
- univariate_categorical(data): Function to generate countplot for each categorical variable. Labeled with count and percentage

**Missing value treatment:**
- created category "other" in 'real_state' variable replacing NaN

**Feature engineering:**
- Droped variable "Unnamed: 0" having no value for modeling
- There are units with cero rooms generating inconsistencies by having multiple bathrooms

**Outlier detection and treatment**
**Preparing data for modeling**
**Any other preprocessing steps**
